# Microsoft Foundry Hosted Agents: Build + Deploy from Notebook

This notebook demonstrates the **hosted agents** execution model (containerized, code-first) using the flow from the Microsoft Learn hosted agents page.

It covers:
1) Local test using the hosting adapter (REST on `localhost:8088`)
2) Build + push Docker image to Azure Container Registry (ACR)
3) Capability host setup (preview requirement)
4) Create hosted agent version (container image) via Azure AI Projects SDK
5) Start/stop/update/list deployment via `az cognitiveservices agent ...`
6) Invoke the hosted agent via Responses-compatible API

> Notes
- This is **not** the classic prompt-based agent. Your existing `AgentsClient` code is included later for contrast.
- Hosted agents require **Docker + ACR**, and appropriate **RBAC**.
- Hosted agents are in preview; region/limits apply.


## 0) Prereqs & Configuration

Fill these values before running.


### ⚠️ Regional Availability — IMPORTANT

**Hosted agents are currently supported only in North Central US.**

If your Foundry project is in any other region, you'll get the error:
> `"Hosted Agents are not enabled in this region."`

**What to do:**
1. **Create a new Foundry project in North Central US**, then run `create_version(...)` there.
2. **Keep your ACR accessible** to that project. ACR can technically be in another region, but cross-region adds latency and potential policy friction — keeping ACR in North Central US (or nearby) is cleaner.

This is a preview limitation and may expand to other regions in the future.

In [1]:
import os
from pathlib import Path

# Minimal config for the early (local) sections.
# Provide Azure/ACR variables when you reach the deploy sections.
APP_DIR = Path("hosted_agent_app")
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT") or os.getenv("PROJECT_ENDPOINT") or ""
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or ""

print("APP_DIR:", APP_DIR)


APP_DIR: hosted_agent_app


## 1) Install Python dependencies

We install:
- `azure-ai-projects` (for hosted agent version creation & invocation)
- `azure-identity`
- Hosting adapter packages (AgentServer) to run locally on `localhost:8088`


In [2]:
# If you're on a clean env, uncomment.
# %pip install -U pip

# Azure AI Projects SDK (docs show a preview/beta in some sections)
# If you need the exact beta: %pip install --pre azure-ai-projects==2.0.0b2
%pip install -U azure-ai-projects azure-identity python-dotenv requests

# Hosting adapter packages (Python)
# These wrap your agent code into a Foundry-compatible HTTP service.
%pip install -U azure-ai-agentserver-core azure-ai-agentserver-agentframework


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2) Azure CLI login + sanity checks

We assume you have:
- `az` installed
- Docker installed

If you want the **azd** path instead of SDK path, you can do it too—but this notebook uses the **SDK + CLI** sequence (it’s the most explicit and debuggable).


In [3]:
import subprocess

def sh(cmd: str) -> str:
    """Run a shell command and return stdout. Raises on failure."""
    print(f"$ {cmd}")
    out = subprocess.check_output(cmd, shell=True, text=True)
    return out.strip()

print(sh("az account show --query name -o tsv"))
print(sh("az account show --query id -o tsv"))
print(sh("az version -o json | python3 -c \"import sys,json; print(json.load(sys.stdin)['azure-cli'])\""))
print(sh("docker --version"))

$ az account show --query name -o tsv
MCAPS-Hybrid-REQ-102171-2024-ozgurguler
$ az account show --query id -o tsv
a20bc194-9787-44ee-9c7f-7c3130e651b6
$ az version -o json | python3 -c "import sys,json; print(json.load(sys.stdin)['azure-cli'])"
2.81.0
$ docker --version
Docker version 29.1.3, build f52814d454


## 3) Write a minimal hosted-agent app (Agent Framework + Hosting Adapter)

This creates a tiny agent service that:
- runs on `localhost:8088`
- exposes `/responses`
- uses your Foundry project endpoint + model deployment

This is the **same mental model** as your existing `AgentsClient` script, but *now the agent is a web service* that can be containerized and hosted.


In [4]:
APP_DIR.mkdir(exist_ok=True)

# Minimal agent service app using the hosting adapter.
# IMPORTANT: this is a *template* because hosting adapter APIs can evolve.
# If you already have your agent-framework code, drop it into this directory.

(APP_DIR / "agent_app.py").write_text(
    '''\
"""Hosted Agent app (Python) - minimal skeleton.

This file is meant to run locally via hosting adapter on localhost:8088.
Later, it gets containerized for Foundry hosted agents.
"""

import os
from dotenv import load_dotenv

# NOTE: The exact import path can vary by package version.
# The hosting adapter concept is: wrap an agent into a Foundry-compatible HTTP service.
from azure.ai.agentserver.agentframework import from_agentframework

# Your agent logic can be as simple or complex as you want.
# Here we define an ultra-minimal "agent" callable.
def my_agent(user_text: str) -> str:
    return f"You said: {user_text}. (Hosted agent skeleton)"

def main():
    load_dotenv()
    # The adapter will start an HTTP server with Foundry-compatible endpoints.
    # Docs conceptually show one-line hosting like: from_langgraph(my_agent).run()
    # For Agent Framework wrapper we use from_agentframework.
    server = from_agentframework(my_agent)
    server.run(host="0.0.0.0", port=8088)

if __name__ == "__main__":
    main()
    ''',
    encoding="utf-8",
)

# .env for local run
(APP_DIR / ".env").write_text(
    f"""\
AZURE_AI_PROJECT_ENDPOINT={PROJECT_ENDPOINT}
MODEL_DEPLOYMENT_NAME={MODEL_DEPLOYMENT_NAME}
""",
    encoding="utf-8",
)

print("Wrote hosted agent app to:", APP_DIR)
print("Files:", [p.name for p in APP_DIR.iterdir()])


Wrote hosted agent app to: hosted_agent_app
Files: ['Dockerfile', 'agent_app.py', '.env']


## 4) Run locally (localhost:8088) + smoke test `/responses`

This validates:
- your agent code runs
- the hosting adapter exposes a Foundry-compatible Responses endpoint


In [5]:
# Start the local server in the background.
# In Jupyter, the simplest approach is to run it in a separate terminal.
# Here we print the command you should run.

print("Run in a terminal:")
print(f"cd {APP_DIR} && python agent_app.py")

print("Then test from another terminal:")
print("curl -s http://localhost:8088/responses -H 'Content-Type: application/json' \\")
print("  -d '{\"input\": {\"messages\": [{\"role\": \"user\", \"content\": \"Where is Seattle?\"}]}}' | jq")


Run in a terminal:
cd hosted_agent_app && python agent_app.py
Then test from another terminal:
curl -s http://localhost:8088/responses -H 'Content-Type: application/json' \
  -d '{"input": {"messages": [{"role": "user", "content": "Where is Seattle?"}]}}' | jq


In [6]:
# Image naming (used for local build and ACR push)
IMAGE_NAME = os.getenv("IMAGE_NAME") or "my-hosted-agent"
IMAGE_TAG = os.getenv("IMAGE_TAG") or "v1"
print("IMAGE:", f"{IMAGE_NAME}:{IMAGE_TAG}")


IMAGE: my-hosted-agent:v1


## 5) Create a Dockerfile + build the image

This containerizes the agent web service so Foundry can host it.


In [7]:
# Minimal Dockerfile. Adjust if you have extra deps.
(APP_DIR / "Dockerfile").write_text(
"""\
FROM python:3.11-slim

WORKDIR /app

# Install runtime deps
COPY . /app

RUN pip install --no-cache-dir -U pip \
    && pip install --no-cache-dir -U python-dotenv azure-identity azure-ai-projects \
    && pip install --no-cache-dir -U azure-ai-agentserver-core azure-ai-agentserver-agentframework

EXPOSE 8088

CMD ["python", "agent_app.py"]
""",
    encoding="utf-8",
)

print("Dockerfile written.")
print("Building docker image...")
sh(f"docker build -t {IMAGE_NAME}:{IMAGE_TAG} {APP_DIR}")
print("Local image built:", f"{IMAGE_NAME}:{IMAGE_TAG}")


Dockerfile written.
Building docker image...
$ docker build -t my-hosted-agent:v1 hosted_agent_app


#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 378B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim


Local image built: my-hosted-agent:v1


#2 DONE 1.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/4] FROM docker.io/library/python:3.11-slim@sha256:158caf0e080e2cd74ef2879ed3c4e697792ee65251c8208b7afb56683c32ea6c
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 1.63kB done
#5 DONE 0.0s

#6 [2/4] WORKDIR /app
#6 CACHED

#7 [3/4] COPY . /app
#7 CACHED

#8 [4/4] RUN pip install --no-cache-dir -U pip     && pip install --no-cache-dir -U python-dotenv azure-identity azure-ai-projects     && pip install --no-cache-dir -U azure-ai-agentserver-core azure-ai-agentserver-agentframework
#8 CACHED

#9 exporting to image
#9 exporting layers done
#9 writing image sha256:12f2093ea291aae85f7f9d38aa60e408f01bc622d5403841e730cb3740250a34 done
#9 naming to docker.io/library/my-hosted-agent:v1 done
#9 DONE 0.0s


## 6) Push image to ACR

Hosted agents require the image to be in Azure Container Registry.


In [8]:
# Reload .env to pick up any changes
from dotenv import load_dotenv
load_dotenv(override=True)

# ACR config (required to push)
# NOTE: ACR_NAME should be just the registry name, e.g. "myregistry" (NOT "myregistry.azurecr.io")
ACR_NAME = os.getenv("ACR_NAME") or ""
if not ACR_NAME:
    raise ValueError("Set ACR_NAME in .env (e.g. ACR_NAME=myregistry, without .azurecr.io)")
ACR_LOGIN_SERVER = f"{ACR_NAME}.azurecr.io"
IMAGE_REF = f"{ACR_LOGIN_SERVER}/{IMAGE_NAME}:{IMAGE_TAG}"
print("IMAGE_REF:", IMAGE_REF)

IMAGE_REF: containervault01.azurecr.io/my-hosted-agent:v1


In [9]:
print("Logging into ACR...")
sh(f"az acr login --name {ACR_NAME}")

print("Tagging + pushing...")
sh(f"docker tag {IMAGE_NAME}:{IMAGE_TAG} {IMAGE_REF}")
sh(f"docker push {IMAGE_REF}")

print("Pushed:", IMAGE_REF)


Logging into ACR...
$ az acr login --name containervault01
Tagging + pushing...
$ docker tag my-hosted-agent:v1 containervault01.azurecr.io/my-hosted-agent:v1
$ docker push containervault01.azurecr.io/my-hosted-agent:v1
Pushed: containervault01.azurecr.io/my-hosted-agent:v1


## 7) Grant ACR pull permissions to the Foundry Project Managed Identity

The docs describe:
- find the project identity principal ID
- assign `Container Registry Repository Reader` on the registry

This step is tenant/RBAC dependent. Below is a **CLI-friendly** skeleton.

> You must fill in or derive the correct principalId for your Foundry project identity.


In [10]:
# ===== YOU MUST SET THIS =====
# This is the managed identity principal object id for your Foundry project.
# In portal: Foundry project -> Identity -> System assigned -> Object (principal) ID
FOUNDRY_PROJECT_PRINCIPAL_ID = os.getenv("FOUNDRY_PROJECT_PRINCIPAL_ID") or ""
if not FOUNDRY_PROJECT_PRINCIPAL_ID:
    print("Set FOUNDRY_PROJECT_PRINCIPAL_ID env var before running this cell.")
else:
    # Get ACR resource ID
    acr_id = sh(f"az acr show -n {ACR_NAME} --query id -o tsv")
    print("ACR ID:", acr_id)

    # Assign pull permissions
    # Role name may vary; docs reference 'Container Registry Repository Reader'
    # If this role name isn't recognized, list ACR roles and pick the correct one.
    sh(
        f"az role assignment create --assignee-object-id {FOUNDRY_PROJECT_PRINCIPAL_ID} "
        f"--assignee-principal-type ServicePrincipal "
        f"--role 'AcrPull' --scope {acr_id}"
    )
    print("Assigned AcrPull to Foundry project identity.")


Set FOUNDRY_PROJECT_PRINCIPAL_ID env var before running this cell.


In [11]:
# Step 1: Get your subscription ID
print("Your subscription ID:")
print(sh("az account show --query id -o tsv"))

Your subscription ID:
$ az account show --query id -o tsv
a20bc194-9787-44ee-9c7f-7c3130e651b6


In [12]:
# Step 2: Find your Foundry account and resource group
# Look for the account that matches your PROJECT_ENDPOINT subdomain
print("Your AI Services accounts (find the one matching your PROJECT_ENDPOINT):\n")
print(sh("az cognitiveservices account list -o table"))
print("\n" + "="*80)
print("HOW TO USE:")
print("1. Look at your PROJECT_ENDPOINT (e.g., https://ACCOUNT_NAME.services.ai.azure.com/...)")
print("2. Find that ACCOUNT_NAME in the 'Name' column above")
print("3. Copy the corresponding 'ResourceGroup' value")
print("="*80)

Your AI Services accounts (find the one matching your PROJECT_ENDPOINT):

$ az cognitiveservices account list -o table
Kind            Location        Name                              ResourceGroup
--------------  --------------  --------------------------------  ---------------------
AIServices      eastus          agent-ai-servicesgezg             rg-openai
AIServices      eastus          agent-ai-servicesjq3h             rg-openai
AIServices      eastus          agent-ai-services7fxt             rg-openai
AIServices      eastus2         ai-ozgurgulerai5658070475260732   rg-ozgurgulerai
AIServices      eastus2         ai-eastus2hubozguler527669401205  rg-openai
AIServices      eastus2         ai-hubx611882637128               rg-openai
AIServices      eastus2         ai-hubxx118150369322              rg-ozgurguler-3950_ai
FormRecognizer  uksouth         docint-ozguler                    rg_xbip
AIServices      northcentralus  ozgur-m3q1pn4n-northcentralus     rg-openai
AIServices   

In [13]:
# Step 3: Extract values from your PROJECT_ENDPOINT automatically
import re
endpoint = PROJECT_ENDPOINT or os.getenv("AZURE_AI_PROJECT_ENDPOINT") or ""

if endpoint:
    # Extract account name from endpoint URL
    match = re.search(r'https://([^.]+)\.services\.ai\.azure\.com/api/projects/([^/]+)', endpoint)
    if match:
        account_name = match.group(1)
        project_name = match.group(2)
        print(f"Detected from your PROJECT_ENDPOINT:")
        print(f"  FOUNDRY_ACCOUNT_NAME = {account_name}")
        print(f"  FOUNDRY_PROJECT_NAME = {project_name}")
        print(f"\nAdd these to your .env file!")
    else:
        print("Could not parse endpoint. Set manually.")
else:
    print("PROJECT_ENDPOINT not set. Run cell 2 first, or set AZURE_AI_PROJECT_ENDPOINT in .env")

Detected from your PROJECT_ENDPOINT:
  FOUNDRY_ACCOUNT_NAME = ozgurguler-7212-resource
  FOUNDRY_PROJECT_NAME = ozgurguler-7212

Add these to your .env file!


In [14]:
# Step 4: Generate .env template with your values
# Copy the output below to your .env file
import re

sub_id = sh("az account show --query id -o tsv")
endpoint = PROJECT_ENDPOINT or os.getenv("AZURE_AI_PROJECT_ENDPOINT") or ""

print("="*60)
print("Add these lines to your .env file:")
print("="*60)
print(f"AZ_SUBSCRIPTION_ID={sub_id}")

if endpoint:
    match = re.search(r'https://([^.]+)\.services\.ai\.azure\.com/api/projects/([^/]+)', endpoint)
    if match:
        account_name = match.group(1)
        project_name = match.group(2)
        # Try to find resource group
        try:
            rg = sh(f"az cognitiveservices account list --query \"[?name=='{account_name}'].resourceGroup\" -o tsv")
            print(f"AZ_RESOURCE_GROUP={rg}")
        except:
            print("AZ_RESOURCE_GROUP=<find from table above>")
        print(f"FOUNDRY_ACCOUNT_NAME={account_name}")
        print(f"FOUNDRY_PROJECT_NAME={project_name}")
print("="*60)

$ az account show --query id -o tsv
Add these lines to your .env file:
AZ_SUBSCRIPTION_ID=a20bc194-9787-44ee-9c7f-7c3130e651b6
$ az cognitiveservices account list --query "[?name=='ozgurguler-7212-resource'].resourceGroup" -o tsv
AZ_RESOURCE_GROUP=rg-ozgurguler-7212
FOUNDRY_ACCOUNT_NAME=ozgurguler-7212-resource
FOUNDRY_PROJECT_NAME=ozgurguler-7212


In [15]:
# Reload .env to pick up any changes you made
from dotenv import load_dotenv
load_dotenv(override=True)

# Azure resource context (required for management-plane + CLI operations)
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID") or ""
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP") or ""
FOUNDRY_ACCOUNT_NAME = os.getenv("FOUNDRY_ACCOUNT_NAME") or ""
FOUNDRY_PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME") or ""

missing = [k for k, v in {
    "AZ_SUBSCRIPTION_ID": SUBSCRIPTION_ID,
    "AZ_RESOURCE_GROUP": RESOURCE_GROUP,
    "FOUNDRY_ACCOUNT_NAME": FOUNDRY_ACCOUNT_NAME,
    "FOUNDRY_PROJECT_NAME": FOUNDRY_PROJECT_NAME,
}.items() if not v]
if missing:
    raise ValueError("Missing required config for deploy sections: " + ", ".join(missing))

print("Azure context OK.")
print(f"  Subscription: {SUBSCRIPTION_ID}")
print(f"  Resource Group: {RESOURCE_GROUP}")
print(f"  Account: {FOUNDRY_ACCOUNT_NAME}")
print(f"  Project: {FOUNDRY_PROJECT_NAME}")

Azure context OK.
  Subscription: a20bc194-9787-44ee-9c7f-7c3130e651b6
  Resource Group: rg-ozgurguler-7212
  Account: ozgurguler-7212-resource
  Project: ozgurguler-7212


### Why do we need an ARM token and Capability Host?

**Capability Host** is a preview requirement for Foundry hosted agents. It tells Azure that your Foundry account is allowed to run containerized agents in a public hosting environment.

**What this cell does:**
1. **Gets an ARM token** — Azure Resource Manager (ARM) is the management plane for Azure. We need an access token to make API calls to create/update Azure resources.
2. **Calls the management API** — We `PUT` a `capabilityHost` resource on your Foundry account with `enablePublicHostingEnvironment=true`.

**Why it's needed:**
- Without this, the hosted agent deployment will fail because Azure doesn't know your account is configured for hosted agents.
- This is a one-time setup per Foundry account (not per project).

In [16]:
# Azure resource context (required for management-plane + CLI operations)
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID") or ""
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP") or ""
FOUNDRY_ACCOUNT_NAME = os.getenv("FOUNDRY_ACCOUNT_NAME") or ""
FOUNDRY_PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME") or ""

missing = [k for k, v in {
    "AZ_SUBSCRIPTION_ID": SUBSCRIPTION_ID,
    "AZ_RESOURCE_GROUP": RESOURCE_GROUP,
    "FOUNDRY_ACCOUNT_NAME": FOUNDRY_ACCOUNT_NAME,
    "FOUNDRY_PROJECT_NAME": FOUNDRY_PROJECT_NAME,
}.items() if not v]
if missing:
    raise ValueError("Missing required config for deploy sections: " + ", ".join(missing))

print("Azure context OK.")


Azure context OK.


In [17]:
# Acquire ARM token
token = sh("az account get-access-token --resource https://management.azure.com/ --query accessToken -o tsv")

# Construct URL
api_version = "2025-10-01-preview"  # from docs
caphost_name = "accountcaphost"     # from docs example

url = (
    f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}"
    f"/resourceGroups/{RESOURCE_GROUP}"
    f"/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_ACCOUNT_NAME}"
    f"/capabilityHosts/{caphost_name}"
    f"?api-version={api_version}"
)

payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "enablePublicHostingEnvironment": True,
    }
}

import requests
resp = requests.put(
    url,
    headers={"content-type": "application/json", "authorization": f"Bearer {token}"},
    json=payload,
)
print("Status:", resp.status_code)
print(resp.text[:2000])


$ az account get-access-token --resource https://management.azure.com/ --query accessToken -o tsv
Status: 500
{"error":{"code":"BadRequest","message":"accountcaphost is invalid: Update of capability is not currently supported. Please delete and recreate with the new configuration.","details":[],"additionalInfo":[{"type":"ComponentName","info":{"value":"managementfrontend"}},{"type":"Correlation","info":{"value":{"operation":"76e87cc39c8337dbd72e9c734698ec51","request":"6ff9564e67830d33"}}},{"type":"Environment","info":{"value":"northcentralus"}},{"type":"Location","info":{"value":"northcentralus"}},{"type":"Time","info":{"value":"2025-12-29T19:51:29.1724274+00:00"}},{"type":"InnerError","info":{"value":{"code":"BadData","innerError":null}}},{"type":"MessageFormat","info":{"value":"An error occurred"}},{"type":"MessageParameters","info":{"value":{"0":"accountcaphost","1":"Update of capability is not currently supported. Please delete and recreate with the new configuration."}}}]}}


## 9) Create Hosted Agent Version (ImageBasedHostedAgentDefinition)

This registers your container image as a hosted agent version in Foundry.

We use `AIProjectClient` and `create_version()` as shown in the docs.


In [18]:
# Hosted agent registration config (required for create_version)
HOSTED_AGENT_NAME = os.getenv("HOSTED_AGENT_NAME") or "ozgur-hosted-agent"
HOSTED_CPU = os.getenv("HOSTED_CPU") or "1"
HOSTED_MEMORY = os.getenv("HOSTED_MEMORY") or "2Gi"
CONTAINER_PROTOCOL_VERSION = os.getenv("CONTAINER_PROTOCOL_VERSION") or "v1"

if not PROJECT_ENDPOINT:
    raise ValueError("Missing PROJECT_ENDPOINT (set AZURE_AI_PROJECT_ENDPOINT or PROJECT_ENDPOINT).")
if not MODEL_DEPLOYMENT_NAME:
    raise ValueError("Missing MODEL_DEPLOYMENT_NAME (or AZURE_OPENAI_DEPLOYMENT_NAME).")
if "IMAGE_REF" not in globals():
    raise ValueError("Missing IMAGE_REF. Run the ACR config cell in step 6 (or set ACR_NAME/IMAGE_NAME/IMAGE_TAG).")

print("PROJECT_ENDPOINT:", PROJECT_ENDPOINT)
print("MODEL_DEPLOYMENT_NAME:", MODEL_DEPLOYMENT_NAME)
print("HOSTED_AGENT_NAME:", HOSTED_AGENT_NAME)
print("IMAGE_REF:", IMAGE_REF)


PROJECT_ENDPOINT: https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212
MODEL_DEPLOYMENT_NAME: gpt-5-nano
HOSTED_AGENT_NAME: ozgur-hosted-agent
IMAGE_REF: containervault01.azurecr.io/my-hosted-agent:v1


In [19]:
# Hosted agents SDK models require azure-ai-projects >= 1.0.0b11 (or 2.0.0b2 for some features)
# If you get ImportError, upgrade: pip install --pre azure-ai-projects>=1.0.0b11

try:
    from azure.identity import DefaultAzureCredential
    from azure.ai.projects import AIProjectClient
    from azure.ai.projects.models import (
        ImageBasedHostedAgentDefinition,
        ProtocolVersionRecord,
        AgentProtocol,
    )
except ImportError as e:
    print(f"Import error: {e}")
    print("\nHosted agent models not found in your SDK version.")
    print("Upgrade with: pip install --pre 'azure-ai-projects>=1.0.0b11'")
    raise

cred = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=cred)

# IMPORTANT:
# Your container might need env vars. Put only what your container expects.
env_vars = {
    "AZURE_AI_PROJECT_ENDPOINT": PROJECT_ENDPOINT,
    "MODEL_DEPLOYMENT_NAME": MODEL_DEPLOYMENT_NAME,
}

agent_version = client.agents.create_version(
    agent_name=HOSTED_AGENT_NAME,
    description="Hosted agent created from notebook",
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[
            ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version=CONTAINER_PROTOCOL_VERSION)
        ],
        cpu=HOSTED_CPU,
        memory=HOSTED_MEMORY,
        image=IMAGE_REF,
        environment_variables=env_vars,
    ),
)

print("Created hosted agent version.")
print("Agent:", agent_version.name if hasattr(agent_version, "name") else HOSTED_AGENT_NAME)
print("Version object:", agent_version)

Created hosted agent version.
Agent: ozgur-hosted-agent
Version object: {'metadata': {}, 'object': 'agent.version', 'id': 'ozgur-hosted-agent:1', 'name': 'ozgur-hosted-agent', 'version': '1', 'description': 'Hosted agent created from notebook', 'created_at': 1767036678, 'definition': {'kind': 'hosted', 'container_protocol_versions': [{'protocol': 'responses', 'version': 'v1'}], 'cpu': '1', 'memory': '2Gi', 'environment_variables': {'AZURE_AI_PROJECT_ENDPOINT': 'https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212', 'MODEL_DEPLOYMENT_NAME': 'gpt-5-nano'}, 'image': 'containervault01.azurecr.io/my-hosted-agent:v1'}}


## 10) Start / Manage the Hosted Agent Deployment (Azure CLI)

Docs show `az cognitiveservices agent start/stop/update/list-versions/show`.

> You need the `agent-version` number. If the SDK object didn’t print it clearly, use `list-versions`.


In [20]:
# Invoke the hosted agent via OpenAI Responses API
# Note: The hosted agent must be in "Running" state (check with list-versions or show commands)

try:
    from azure.ai.projects.models import AgentReference
except ImportError:
    print("AgentReference not available - upgrade azure-ai-projects")
    raise

openai_client = client.get_openai_client()

# Create the agent reference directly (no need to retrieve first)
agent_ref = AgentReference(name=HOSTED_AGENT_NAME, version=AGENT_VERSION_TO_START)

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": agent_ref.as_dict()},
)

print(resp.output_text)

NameError: name 'AGENT_VERSION_TO_START' is not defined

In [ ]:
# ===== SET THE VERSION YOU WANT TO START =====
AGENT_VERSION_TO_START = os.getenv("AGENT_VERSION_TO_START") or "1"

print(sh(
    f"az cognitiveservices agent start "
    f"--account-name {FOUNDRY_ACCOUNT_NAME} "
    f"--project-name {FOUNDRY_PROJECT_NAME} "
    f"--name {HOSTED_AGENT_NAME} "
    f"--agent-version {AGENT_VERSION_TO_START} "
    f"-o json"
))


In [ ]:
# Invoke the hosted agent via OpenAI Responses API
# Note: The hosted agent must be in "Running" state (check with az cognitiveservices agent show)

try:
    from azure.ai.projects.models import AgentReference
except ImportError:
    print("AgentReference not available - upgrade azure-ai-projects")
    raise

openai_client = client.get_openai_client()

# Create the agent reference directly (no need to retrieve the agent first)
agent_ref = AgentReference(name=HOSTED_AGENT_NAME, version=AGENT_VERSION_TO_START)

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": agent_ref.as_dict()},
)

print(resp.output_text)

In [ ]:
# Invoke the hosted agent via OpenAI Responses API
# Note: The hosted agent must be in "Running" state

# Ensure AGENT_VERSION_TO_START is set (default to "1" if not defined)
if "AGENT_VERSION_TO_START" not in dir():
    AGENT_VERSION_TO_START = os.getenv("AGENT_VERSION_TO_START") or "1"

try:
    from azure.ai.projects.models import AgentReference
except ImportError:
    print("AgentReference not available - upgrade azure-ai-projects")
    raise

openai_client = client.get_openai_client()

# Create agent reference directly - no need to call retrieve()
agent_ref = AgentReference(name=HOSTED_AGENT_NAME, version=AGENT_VERSION_TO_START)

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": agent_ref.as_dict()},
)

print(resp.output_text)

## 12) (Optional) Keep your classic prompt-based agent client for comparison

This is your original approach (agent definition executed inside the classic Foundry agent runtime). It does **not** deploy code.

Hosted agents = your code deployed as a container.


In [ ]:
# --- Your original classic script (kept as reference) ---
import os
from dotenv import load_dotenv

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

load_dotenv()

def classic_main() -> None:
    project_endpoint = os.getenv("PROJECT_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    if not project_endpoint:
        raise ValueError("Missing PROJECT_ENDPOINT (or AZURE_AI_PROJECT_ENDPOINT).")

    model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    if not model_deployment:
        raise ValueError("Missing MODEL_DEPLOYMENT_NAME (or AZURE_OPENAI_DEPLOYMENT_NAME).")

    credential = DefaultAzureCredential()
    agents_client = AgentsClient(endpoint=project_endpoint, credential=credential)

    with agents_client:
        code_interpreter = CodeInterpreterTool()

        agent = agents_client.create_agent(
            model=model_deployment,
            name="Quickstart",
            instructions="Be concise.",
            tools=code_interpreter.definitions,
            tool_resources=code_interpreter.resources,
        )
        print(f"Created classic agent: {agent.id}")

        thread = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="Write a haiku about Azure AI Foundry.",
        )

        run = agents_client.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent.id,
        )
        print(f"Run status: {run.status}")

        messages = agents_client.messages.list(thread_id=thread.id)
        for m in messages:
            print(f"{m.role}: {m.content}")

# classic_main()


## 13) Cleanup

To stop / delete the hosted deployment:

- `az cognitiveservices agent stop ...`
- `az cognitiveservices agent delete-deployment ...`
- `az cognitiveservices agent delete ...`

And optionally delete ACR image/tag.
